In [1]:
!apt-get update && apt-get install -y openmpi-bin libopenmpi-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,007 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,668 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal In

In [2]:
%%writefile exp8.c
#include <stdio.h>
#include <math.h>
#include <mpi.h>
#include <omp.h>

double leibniz_pi(int n, int rank, int size) {
    double pi = 0.0;
    int sign = (rank % 2 == 0) ? 1 : -1;

    #pragma omp parallel for reduction(+:pi)
    for (int i = rank; i < n; i += size) {
        double term = 1.0 / (2*i + 1);
        pi += sign * term;
        sign *= -1;
    }

    double global_pi = 0.0;
    MPI_Allreduce(&pi, &global_pi, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
    return global_pi * 4;
}

double cube_volume(double a, double pi) {
    return pow(a, 3);
}

double cuboid_volume(double l, double w, double h, double pi) {
    return l * w * h;
}

double tetrahedron_volume(double b, double h, double pi) {
    return (1.0/3.0) * b * h;
}

double cylinder_volume(double r, double h, double pi) {
    return pi * pow(r, 2) * h;
}

int main(int argc, char** argv) {
    int n = 1000000;
    int rank, size;
    double pi;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    double start_time = MPI_Wtime();

    pi = leibniz_pi(n, rank, size);

    double end_time = MPI_Wtime();
    double elapsed_time = end_time - start_time;

    if (rank == 0) {
        printf("Pi = %.16f\n", pi);
        printf("Leibniz series terms = %d\n", n);
        printf("Elapsed time = %.4f seconds\n", elapsed_time);

        // Calculate volumes
        double a = 2.0;
        double l = 3.0;
        double w = 4.0;
        double h = 5.0;
        double r = 2.0;

        printf("Volume of Cube = %.2f\n", cube_volume(a, pi));
        printf("Volume of Cuboid = %.2f\n", cuboid_volume(l, w, h, pi));
        printf("Volume of Tetrahedron = %.2f\n", tetrahedron_volume(0.5*l*w, h, pi));
        printf("Volume of single side closed Cylinder = %.2f\n", cylinder_volume(r, h, pi));
    }

    MPI_Finalize();
    return 0;
}



Writing exp8.c


In [3]:
!mpicc -fopenmp -o exp8 exp8.c -lm

In [4]:
!mpirun -np 3 --allow-run-as-root --oversubscribe ./exp8 

Pi = -4.0885805364084034
Leibniz series terms = 1000000
Elapsed time = 0.0202 seconds
Volume of Cube = 8.00
Volume of Cuboid = 60.00
Volume of Tetrahedron = 10.00
Volume of single side closed Cylinder = -81.77
